In [1]:
import numpy as np
import pandas as pd 
import glob
import pickle
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

In [2]:
with open('news', 'r') as f:
    text = f.read()
    news = text.split("\n\n")
    
    count = {"sport": 0, "world": 0, "us": 0, "business": 0, "health": 0, "entertainment": 0, "sci_tech": 0}
    for news_item in news:
        lines = news_item.split("\n")
        if len(lines) == 7:
                file_to_write = open('data/' + lines[6] + '/' + str(count[lines[6]]) + '.txt', 'w+')
                count[lines[6]] = count[lines[6]] + 1
                file_to_write.write(news_item)  # python will convert \n to os.linesep
                file_to_write.close()

        else:
                print("end of file reached")

                
keys= count.keys()
values = count.values()
plt.figure(figsize=(10,5))
plt.bar(keys, values, width=0.75)
plt.xlabel('Category', fontsize=14)
plt.ylabel('Number of Articles', fontsize=14)
plt.title('News Article Category Distribution for TagMyNews Data', fontsize=16)
       

FileNotFoundError: [Errno 2] No such file or directory: 'data/sport/0.txt'

In [ ]:
with open('news', 'r') as f:
    text = f.read()
    news = text.split("\n\n")
    
    count = {"nyt": 0, "ut": 0, "reuters": 0}
    for news_item in news:
        lines = news_item.split("\n")
        if len(lines) == 7:
            count[lines[5]] = count[lines[5]] + 1

keys= count.keys()
values = count.values()
plt.figure(figsize=(10,5))
plt.bar(keys, values, width=0.75)
plt.xlabel('Source', fontsize=14)
plt.ylabel('Number of Articles', fontsize=14)
plt.title('News Article Source Distribution for TagMyNews Data', fontsize=16)

In [ ]:
keys= count.keys()
values = count.values()
plt.figure(figsize=(10,5))
plt.bar(keys, values, width=0.75)
plt.xlabel('Category', fontsize=14)
plt.ylabel('Number of Articles', fontsize=14)
plt.title('News Article Category Distribution for TagMyNews Data', fontsize=16)

In [ ]:
category_list = ["sport", "world", "us", "business", "health", "entertainment", "sci_tech"]
directory_list = ["data/sport/*.txt", "data/world/*.txt","data/us/*.txt","data/business/*.txt","data/health/*.txt","data/entertainment/*.txt","data/sci_tech/*.txt"]

text_files = list(map(lambda x: glob.glob(x), directory_list))
text_files = [item for sublist in text_files for item in sublist]

news_data = []


for t in text_files:
    f = open(t, 'r')
    f = f.read()
    t = f.split('\n')
    news_data.append({'data' : t[0] + ' ' + t[1] , 'flag' : category_list.index(t[6])})

In [ ]:
news_data = pd.DataFrame(news_data, columns=['data', 'flag'])
news_data.to_csv("news_data.csv", sep=',', encoding='utf-8')
print(news_data.data.size)

In [ ]:
news_data.describe()

In [ ]:
news_data.info()

In [ ]:
list(news_data)

In [ ]:
news_data.isnull().sum()


In [ ]:
#GET VECTOR COUNT
count_vect = CountVectorizer(stop_words='english', analyzer='word', token_pattern=r"(?u)\b\w\w+\b")
X_train_counts = count_vect.fit_transform(news_data.data)

#SAVE WORD VECTOR
pickle.dump(count_vect.vocabulary_, open("count_vector.pkl","wb"))

In [ ]:
X_train_counts.shape

In [ ]:
count_vect.vocabulary_

In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer

#TRANSFORM WORD VECTOR TO TF IDF
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

#SAVE TF-IDF
pickle.dump(tfidf_transformer, open("tfidf.pkl","wb"))

In [ ]:
X_train_tfidf

In [ ]:
# Multinomial Naive Bayes

from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split

#clf = MultinomialNB().fit(X_train_tfidf, news_data.flag)
X_train, X_test, y_train, y_test = train_test_split(X_train_tfidf, news_data.flag, test_size=0.25, random_state=42)
clf = MultinomialNB().fit(X_train, y_train)

#SAVE MODEL
pickle.dump(clf, open("nb_model.pkl", "wb"))

In [ ]:
import pickle
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

category_list = ["sport", "world", "us", "business", "health", "entertainment", "sci_tech"]

docs_new = "Messi in football"
docs_new = [docs_new]

#LOAD MODEL
loaded_vec = CountVectorizer(vocabulary=pickle.load(open("count_vector.pkl", "rb")))
loaded_tfidf = pickle.load(open("tfidf.pkl","rb"))
loaded_model = pickle.load(open("nb_model.pkl","rb"))

X_new_counts = loaded_vec.transform(docs_new)
X_new_tfidf = loaded_tfidf.transform(X_new_counts)
predicted = loaded_model.predict(X_new_tfidf)

print(category_list[predicted[0]])



In [ ]:
predicted = loaded_model.predict(X_test)
result_bayes = pd.DataFrame( {'true_labels': y_test,'predicted_labels': predicted})
result_bayes.to_csv('res_bayes.csv', sep = ',')

for predicted_item, result in zip(predicted, y_test):
    print(category_list[predicted_item], ' - ', category_list[result])



In [ ]:
from sklearn.metrics import classification_report, confusion_matrix  

def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    import itertools
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)
    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()
    plt.show()

confusion_mat = confusion_matrix(y_test,predicted)
plt.figure(figsize=[8.5,7])
plot_confusion_matrix(confusion_mat,classes=category_list,title='Confusion Matrix of Multinomial Naive Bayes Model')

In [ ]:
print (classification_report(y_test, predicted))

In [ ]:
from sklearn.neural_network import MLPClassifier

clf_neural = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(15,), random_state=1)

X_train, X_test, y_train, y_test = train_test_split(X_train_tfidf, news_data.flag, test_size=0.25, random_state=42)

clf_neural.fit(X_train, y_train)

In [ ]:
pickle.dump(clf_neural, open("softmax.pkl", "wb"))

In [ ]:
predicted = clf_neural.predict(X_test)
result_softmax = pd.DataFrame( {'true_labels': y_test,'predicted_labels': predicted})
result_softmax.to_csv('res_softmax.csv', sep = ',')

for predicted_item, result in zip(predicted, y_test):
    print(category_list[predicted_item], ' - ', category_list[result])


In [ ]:
from sklearn.metrics import classification_report, confusion_matrix  

confusion_mat = confusion_matrix(y_test,predicted)
plt.figure(figsize=[8.5,7])
plot_confusion_matrix(confusion_mat,classes=category_list,title='Confusion Matrix of Neural Networks with SoftMax Layer ')

In [ ]:
print (classification_report(y_test, predicted))

In [ ]:
from sklearn import svm
clf_svm = svm.LinearSVC()
X_train, X_test, y_train, y_test = train_test_split(X_train_tfidf, news_data.flag, test_size=0.25, random_state=42)
clf_svm.fit(X_train_tfidf, news_data.flag)
pickle.dump(clf_svm, open("svm.pkl", "wb"))

In [ ]:
predicted = clf_svm.predict(X_test)
result_svm = pd.DataFrame( {'true_labels': y_test,'predicted_labels': predicted})
result_svm.to_csv('res_svm.csv', sep = ',')
for predicted_item, result in zip(predicted, y_test):
    print(category_list[predicted_item], ' - ', category_list[result])

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix  

confusion_mat = confusion_matrix(y_test,predicted)
print(confusion_mat)

In [ ]:
print (classification_report(y_test, predicted))